In [1]:
# HIDE CODE
from dateutil.relativedelta import relativedelta
from joblib import dump, load
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
matplotlib.rcParams.update({'font.size': 12})


import warnings
import sys
import os
sys.path.append('/home/server/gli-data-science/')
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

import os
import ds_db
import helper_db
from helper import transform_to_rupiah, rupiah_format

import pickle
import pandas as pd
import numpy as np
import psycopg2

from datetime import datetime, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
pd.options.mode.chained_assignment = None  # default='warn'


from sqlalchemy import event,create_engine,types

driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )

### ide kasar
> 
* daily job to loop all member alfagift 10 juta
* compute feature trx voucher used, # of trx, # of sales
* compute feature event app site open / view product / atc
* updating each feature and re predict churn feature
* lets play in month - 3 for each define trx and event feature

In [2]:
q = '''
select 
	atvd_id, atvd_member, atvd_registration_code, atvd_blast_date, atvd_virgo_date 
from 
	alfagift.alfagift_targeted_voucher_det
where 
	atvd_amtv_id  = '8'--churn
	and atvd_blast_date < '2022-03-07'
'''

connection_voucher = psycopg2.connect(
    host="35.220.168.98",
    database="postgres",
    user="ferina",
    password="nw9f20d")

df_churn = pd.read_sql_query(q, connection_voucher)
connection_voucher.close()

In [3]:



engine = create_engine(engine_stmt)

df = df_churn[['atvd_member', 'atvd_registration_code']].astype(str)
dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
df.to_sql('temp_voucher_', engine, index=False, if_exists="replace", dtype=dtyp)

engine.dispose()


In [25]:
q = '''
SELECT 
rto.TRO_PONTA_ID, 
rto.TRO_PLU, 
rto.TRO_QTY, 
rto.TRO_NET, 
rto.TRO_VOUCHER_CODE, 
rto.TRO_DATE
FROM TEMP_VOUCHER_ tv
LEFT JOIN RPT_TRANS_OSHOP rto ON rto.TRO_PONTA_ID = tv.ATVD_MEMBER 
WHERE rto.TRO_DATE BETWEEN '1-sep-21' AND '15-mar-22'

'''
con = ds_db.connect_alfabi()
df_trx = pd.read_sql_query(q, con)

con.close()

You are connected


In [7]:
q = '''
SELECT 
	tv.ATVD_MEMBER,
	gua.GUA_ACCOUNT_CARD_ADJ,
	gua.GUA_UNINSTALL_DATE,
	gua.GUA_LAST_SEEN
FROM TEMP_VOUCHER_ tv 
	INNER JOIN GLI_UNINSTALL_ALFAGIFT gua ON tv.ATVD_MEMBER = gua.GUA_ACCOUNT_CARD_ADJ
where 
	gua_status = 'Y'
	AND trunc(GUA_UNINSTALL_DATE) BETWEEN '01-jul-21' AND '28-feb-22'

'''

con = ds_db.connect_alfabi()
df_uninstall = pd.read_sql_query(q, con)

con.close()

You are connected


In [9]:
df_uninstall.shape

(1050, 4)

In [11]:
df_uninstall[df_uninstall['ATVD_MEMBER'] == '9990013481402300']

,ATVD_MEMBER,GUA_ACCOUNT_CARD_ADJ,GUA_UNINSTALL_DATE,GUA_LAST_SEEN
1044,9990013481402300,9990013481402300,2022-02-26 16:42:18,2022-02-13 02:19:52


In [10]:
q = '''

SELECT * 
FROM 
    RPT_TRANS_OSHOP rto 
WHERE 
    rto.tro_members = '9990013481402300'
    AND rto.tro_date BETWEEN '01-jul-21' AND '28-feb-22'

'''

con = ds_db.connect_alfabi()
df_uninstall_check = pd.read_sql_query(q, con)
con.close()
df_uninstall_check

You are connected


,TRO_DATE,TRO_DATE_SHIP,TRO_KD_STORE,TRO_BRANCH_CODE,TRO_MEMBERS,TRO_CUST_ID,TRO_NO_REF,TRO_NO_ORDER,TRO_PAYMENT_METHOD,TRO_PLU,TRO_QTY,TRO_NET,TRO_GM,TRO_DISC,TRO_DH_SPESIAL,TRO_DATE_ORDER,TRO_SHIP_METHOD,TRO_SHIP_METHOD_NAME,TRO_KODE_KASIR,TRO_NO_BON,TRO_JAM_TRANSAKSI,TRO_TYPE,TRO_PRICE,TRO_POS_NOREF,TRO_CREATE_DATE,TRO_UPDATE_DATE,TRO_PONTA_ID,TRO_MCW_ID,TRO_VOUCHER_CODE,TRO_VOUCHER_USAGE,TRO_TAG,TRO_STATUS_RETUR,TRO_CART_ID,TRO_OTHER_INFO,TRO_AVG_COST,TRO_STATUS_SAPA,TRO_APPLICATION_ID,TRO_CART_SOURCE,TRO_CART_PROMO_PAGE_ID,TRO_CART_PROMO_PAGE_DESC,TRO_TRIGGER_OPEN_APP,TRO_TRIGGER_OPEN_APP_DESC
0,2021-12-05,2021-12-05,CB40,CZ01,9990013481402300,2174771,S-211205-AGXGBZM,O-211205-AGDQMFS,GO-PAY,103001,1,181.8,0.0,0.0,200.0,2021-12-05 01:16:48,0,0,105,5153,2021-12-05 01:31:47,O,200.0,S-211205-AGXGBZM,2021-12-07 09:48:05,2022-01-06 10:50:54,9990013481402300,25748291,None,0,L,N,15916895,ALFAGIFT,250.9,N,905,None,None,None,None,None
1,2021-12-05,2021-12-05,CB40,CZ01,9990013481402300,2174771,S-211205-AGXGBZM,O-211205-AGDQMFS,GO-PAY,192730,2,66363.6,6102.0,0.0,0.0,2021-12-05 01:16:48,0,0,105,5153,2021-12-05 01:31:47,O,36500.0,S-211205-AGXGBZM,2021-12-07 09:48:05,2022-01-06 10:50:54,9990013481402300,25748291,None,0,T,N,15916895,ALFAGIFT,30130.7,N,905,RECOMMENDATION_HOME,None,None,None,None
2,2022-01-06,2022-01-06,C593,CZ01,9990013481402300,2174771,S-220106-AGJMFVG,O-220106-AGTHNHV,GO-PAY,103001,1,181.8,0.0,0.0,200.0,2022-01-06 06:43:36,0,0,102,2464,2022-01-06 07:05:12,O,200.0,S-220106-AGJMFVG,2022-01-08 09:18:25,2022-02-07 11:02:33,9990013481402300,25748291,None,0,L,N,17945393,ALFAGIFT,250.9,N,905,None,None,None,None,None
3,2022-01-06,2022-01-06,C593,CZ01,9990013481402300,2174771,S-220106-AGJMFVG,O-220106-AGTHNHV,GO-PAY,192730,2,68909.1,3690.3,0.0,0.0,2022-01-06 06:43:36,0,0,102,2464,2022-01-06 07:05:12,O,37900.0,S-220106-AGJMFVG,2022-01-08 09:18:25,2022-02-07 11:02:33,9990013481402300,25748291,None,0,T,N,17945393,ALFAGIFT,32609.4,N,905,BY_KEYWORD,None,None,None,None


In [28]:
df_trx[df_trx['TRO_DATE'].dt.strftime('%b-%y') == 'Sep-21']['TRO_PONTA_ID'].nunique()

61

In [31]:
# def is_unique(s):
#     a = s.to_numpy() # s.values (pandas<0.24)
#     return (a[0] == a).all()

# is_unique(df['counts'])

In [ ]:
df_pv_ = pd.read_csv('./feature/pv_31Jan22.csv')
df_pv_['EVENT_TIME'] = pd.to_datetime(df_pv_['EVENT_TIME']).dt.strftime('%Y-%m')

df_pv = df_pv_.groupby(['TRO_MEMBERS','EVENT_TIME']).agg(PV=('COUNT_VIEW_PRODUCT','sum'))\
        .unstack(level=1).fillna(0).reset_index()

df_pv.columns = [' '.join(col).strip().upper() for col in df_pv.columns.values]

In [ ]:
df_pv.head()

In [ ]:
df_re_g_u = pd.read_csv('./feature/rating_2022-02-01.csv')

In [ ]:
df_re_g_u.head()

In [ ]:
# HIDE CODE

## USING ORDER
import glob

df_tto = []
for end_date in pd.date_range('2021-09-01', '2022-03-08', freq='M'):
    start_date = end_date.replace(day=1)
    end_date_save = start_date + relativedelta(months=1)
    
    start_date_str = start_date.strftime('%d%b%y')
    end_date_str = end_date_save.strftime('%d%b%y')
    
    print(start_date_str, end_date_str)
    
    pf = '/home/server/gli-data-science/data/trans_oshop_test/{}_{}.csv'.format(start_date_str, end_date_str)
    df_tto.append(pd.read_csv(pf, sep='\t', dtype='object'))
    
df_tto = pd.concat(df_tto)

df_tto = df_tto[df_tto['TRO_MEMBERS'] != 'N']
df_tto['TRO_DATE'] = pd.to_datetime(df_tto['TRO_DATE'])
df_tto['TRO_NET'] = df_tto['TRO_NET'].astype('float', errors='ignore').astype('int', errors='ignore')


df_tto['TRO_QTY'] = df_tto['TRO_QTY'].astype(int)

In [ ]:
# ## USE THIS FOR GET PV DATA

# df_mem_push = df_tto[['TRO_MEMBERS']].drop_duplicates()
# from sqlalchemy import event,create_engine,types



# driver = 'cx_oracle'
# server = '10.234.152.61' 
# database = 'alfabi' 
# username = 'report' 
# password = 'justd0it'
# engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )

# engine = create_engine(engine_stmt)

# df = df_mem_push.astype(str)
# dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
# df.to_sql('temp_churn', engine, index=False, if_exists="replace", dtype=dtyp)

# engine.dispose()





In [ ]:
from sqlalchemy import event,create_engine,types


driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % ( username, password, server, database )



engine = create_engine(engine_stmt)
q = '''
    SELECT AVG(NUM_MEMBER) AS AVG_NUM_MEMBER
    FROM TEMP_SALES_PROMO_ALFAGIFT tspa 
    WHERE tspa.PLU = {}
    AND tspa.TYPE = {}

'''.format(41091, 201)
con = engine.connect()
try:
    res_avg = pd.read_sql_query(q,con)
except Exception as e:
    if is_debug:
        print(e)
    pass
con.close()
engine.dispose()

res_avg['avg_num_member'][0]

In [ ]:
res_avg.

In [ ]:
df_tto['TRO_VOUCHER_USAGE'] = df_tto['TRO_VOUCHER_USAGE']\
                            .astype('float', errors='ignore').astype('int', errors='ignore')
df_tto['TRO_DH_SPESIAL'] = df_tto['TRO_DH_SPESIAL']\
                            .astype('float', errors='ignore').astype('int', errors='ignore')


df_tto_sel = df_tto[(df_tto['TRO_DATE'] >= '2021-10-01') \
            & (df_tto['TRO_DATE'] <= '2022-01-31')]

In [ ]:
df_sla = pd.read_csv('./feature/sla_31Jan22.csv')
df_sla['ATT_SEND_DATE_TOSTORE'] = pd.to_datetime(df_sla['ATT_SEND_DATE_TOSTORE'])
df_sla['SLA'] = np.where(df_sla['SLA_NORM']< 0, 20, df_sla['SLA_NORM'])

df_tto_sel = pd.merge(df_tto_sel, df_sla, left_on='TRO_NO_ORDER', right_on='ATT_ORDER_ID', how='left')

col_drop = [
    'ATT_ORDER_ID',
    'ATT_ORDER_DATE',
    'ATT_DELIVERY_DATE',
    'ATT_SEND_DATE_TOSTORE',
    'SLA_NORM'        
]
df_tto_sel = df_tto_sel.drop(col_drop,1)

## create feature
df_tto_sel['SLA'] = df_tto_sel['SLA'].fillna(20)
df_tto_sel['SLA_LATE'] = np.where(df_tto_sel['SLA'] > 60, 1,0)

In [ ]:
q = '''
SELECT 
	tc.TRO_MEMBERS, 
	(TO_DATE('2022/02/08', 'yyyy/mm/dd')) - zt.tgl_registrasi AS SINCE_REGIS
FROM 
	TEMP_CHURN tc
LEFT JOIN (
	select 
		char1 no_member, char4 jenis_kelamin, char5 status_nikah, date1 tgl_lahir, date2 tgl_registrasi
	from 
		report.ztampungan
	where 
		pk='MEMBERALFAGIFT2'
) zt ON zt.no_member = tc.TRO_MEMBERS 

'''

con = ds_db.connect_alfabi()

df_regis = pd.read_sql_query(q, con)

con.close()

In [ ]:
df_regis['SINCE_REGIS'] = df_regis['SINCE_REGIS'].fillna('-1')

In [ ]:
df_tto_sel_g = df_tto_sel.groupby(['TRO_MEMBERS','TRO_DATE_MONTH','TRO_NO_ORDER'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_DH_SPESIAL':'sum',
                              'TRO_NO_ORDER':'nunique',
                              'SLA':'first',
                              'SLA_LATE':'first',
                              'TRO_VOUCHER_USAGE':'sum',
                              'TRO_PAYMENT_METHOD':'first'
                              ''
                             })
df_tto_sel_g = df_tto_sel_g.groupby(['TRO_MEMBERS','TRO_DATE_MONTH'])\
                        .agg({'TRO_NET':'sum',
                              'TRO_DH_SPESIAL':'sum',
                              'TRO_NO_ORDER':'sum',
                              'SLA':'sum',
                              'SLA_LATE':'sum',
                              'TRO_VOUCHER_USAGE':'sum',
                              'TRO_PAYMENT_METHOD':'nunique'
                             })

## unstack all feature in oshop

In [ ]:
df_tto_sel_g_u = df_tto_sel_g.unstack(level=1).fillna(0).reset_index()
df_tto_sel_g_u.columns = [' '.join(col).strip().upper() for col in df_tto_sel_g_u.columns.values]

## merge with rating

In [ ]:
df_re_g_u['TRO_MEMBERS'] = df_re_g_u['TRO_MEMBERS'].astype(str)
df_tto_re = pd.merge(df_tto_sel_g_u, df_re_g_u, on='TRO_MEMBERS', how='left').fillna(0)

## merge with product view

In [ ]:
df_pv['TRO_MEMBERS'] = df_pv['TRO_MEMBERS'].astype(str)
df_tto_re_pv = pd.merge(df_tto_re, df_pv, on='TRO_MEMBERS', how='left').fillna(0)

## merge with regis

In [ ]:
df_tto_re_pv = pd.merge(df_tto_re_pv, df_regis, on='TRO_MEMBERS')
df_tto_re_pv = df_tto_re_pv[df_tto_re_pv['SINCE_REGIS'].astype(int) > 0]

In [ ]:
df_tto_re_pv.head()

In [ ]:
df_tto_re_pv['NET 11-10'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-10']
df_tto_re_pv['NET 10-09'] = df_tto_re_pv['TRO_NET 2021-10'] - df_tto_re_pv['TRO_NET 2021-09']
df_tto_re_pv['NET 11-09'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-09']

df_tto_re_pv['STRUK 11-10'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']
df_tto_re_pv['STRUK 10-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-10'] - df_tto_re_pv['TRO_NO_ORDER 2021-09']
df_tto_re_pv['STRUK 11-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-09']

df_tto_re_pv['PV 11-10'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-10']
df_tto_re_pv['PV 10-09'] = df_tto_re_pv['PV 2021-10'] - df_tto_re_pv['PV 2021-09']
df_tto_re_pv['PV 11-09'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-09']

df_tto_re_pv['SLA 11-10'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-10']
df_tto_re_pv['SLA 10-09'] = df_tto_re_pv['SLA 2021-10'] - df_tto_re_pv['SLA 2021-09']
df_tto_re_pv['SLA 11-09'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-09']

df_tto_re_pv['BASKET 11'] = df_tto_re_pv['TRO_NET 2021-11']/df_tto_re_pv['TRO_NO_ORDER 2021-11']
df_tto_re_pv['BASKET 10'] = df_tto_re_pv['TRO_NET 2021-10']/df_tto_re_pv['TRO_NO_ORDER 2021-10']
df_tto_re_pv['BASKET 09'] = df_tto_re_pv['TRO_NET 2021-09']/df_tto_re_pv['TRO_NO_ORDER 2021-09']

df_tto_re_pv['BASKET AVG'] = (df_tto_re_pv['BASKET 11'] + df_tto_re_pv['BASKET 10'] + df_tto_re_pv['BASKET 09']) / 3

In [ ]:
# print(df_ins[df_ins['label'] == 1]['NET_11-10'].median())
# print(df_ins[df_ins['label'] == 1]['NET_10-09'].median())
# print(df_ins[df_ins['label'] == 1]['NET_11-09'].median())
# print(df_ins[df_ins['label'] == 1]['STRUK_11-10'].mean())
# print(df_ins[df_ins['label'] == 1]['STRUK_10-09'].mean())
# print(df_ins[df_ins['label'] == 1]['STRUK_11-09'].mean())

## PCA Explained

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
condition_2_3 = (
((df_tto_re_pv['TRO_NET 2021-09'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0)) | \
    
((df_tto_re_pv['TRO_NET 2021-09'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0)) |
    
((df_tto_re_pv['TRO_NET 2021-09'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0))
)

df_tto_re_pv['label'] = np.where(condition_2_3, 1, 0)


X = df_tto_re_pv.drop(
[
'TRO_MEMBERS',
'TRO_NET 2021-12', 
'TRO_NO_ORDER 2021-12',
'SLA 2021-12',
'SLA_LATE 2021-12',
'RATING_BELOW 2021-12',
'RATING_AVG 2021-12',
'PV 2021-12',
'TRO_VOUCHER_USAGE 2021-12',
'TRO_PAYMENT_METHOD 2021-12',
'label'
], 1
)
y = df_tto_re_pv[['label']]
matrix = X.iloc[:,0:].copy()
pca = PCA()
pca.fit(matrix)
pca_samples = pca.transform(matrix)

fig, ax = plt.subplots(figsize=(14, 7), dpi=170)
sns.set(font_scale=1)
plt.step(range(matrix.shape[1]), pca.explained_variance_ratio_.cumsum(), where='mid',
         label='cumulative explained variance')
sns.barplot(np.arange(1,matrix.shape[1]+1), pca.explained_variance_ratio_, alpha=0.5, color = 'g',
            label='individual explained variance')
plt.xlim(0, 100)

ax.set_xticklabels([s if int(s.get_text())%2 == 0 else '' for s in ax.get_xticklabels()])

plt.ylabel('Explained variance', fontsize = 14)
plt.xlabel('Principal components', fontsize = 14)
plt.legend(loc='upper left', fontsize = 13);

## Hist Boosting

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier


from sklearn.model_selection import StratifiedKFold
condition_2_3 = (
((df_tto_re_pv['TRO_NET 2021-09'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0)) | \
    
((df_tto_re_pv['TRO_NET 2021-09'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0)) |
    
((df_tto_re_pv['TRO_NET 2021-09'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0))
)

df_tto_re_pv['label'] = np.where(condition_2_3, 1, 0)


X = df_tto_re_pv.drop(
[
'TRO_MEMBERS',
'TRO_NET 2021-12', 
'TRO_DH_SPESIAL 2021-12',
'TRO_NO_ORDER 2021-12',
'SLA 2021-12',
'SLA_LATE 2021-12',
'RATING_BELOW 2021-12',
'RATING_AVG 2021-12',
'PV 2021-12',
'TRO_VOUCHER_USAGE 2021-12',
'TRO_PAYMENT_METHOD 2021-12',
'label'
], 1
)
y = df_tto_re_pv[['label']]



skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, y):
    print("="*50)
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]

    
    ## applying standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_train.columns)
    


    clf_ridge = HistGradientBoostingClassifier()
    clf_ridge.fit(X_train, y_train) 
    pred = clf_ridge.predict(X_test)

    print(classification_report(y_test, pred))
    fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
    ConfusionMatrixDisplay.from_predictions(y_test, pred, 
                                            cmap='Blues', display_labels= ['not_churn', 'churn'], 
                                            ax = ax, values_format = 'd')



## Random Forest

### not oversampling version without threshold 2/3

In [ ]:
from sklearn.model_selection import StratifiedKFold
condition_2_3 = (
((df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-01'] == 0)) | \
    
((df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-01'] == 0)) |
    
((df_tto_re_pv['TRO_NET 2021-10'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] > 0) & \
(df_tto_re_pv['TRO_NET 2022-01'] == 0))
)

df_tto_re_pv['label'] = np.where(condition_2_3, 1, 0)


X = df_tto_re_pv.drop(
[
'TRO_MEMBERS',
'TRO_NET 2022-01', 
'TRO_DH_SPESIAL 2022-01',
'TRO_NO_ORDER 2022-01',
'SLA 2022-01',
'SLA_LATE 2022-01',
'RATING_BELOW 2022-01',
'RATING_AVG 2022-01',
'PV 2022-01',
'TRO_VOUCHER_USAGE 2022-01',
'TRO_PAYMENT_METHOD 2022-01',
'label'
], 1
)
y = df_tto_re_pv[['label']]



# skf = StratifiedKFold(n_splits=5)
# for train_index, test_index in skf.split(X, y):
#     print("="*50)
#     X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
#     y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]


    
#     ## applying standard scaler
#     scaler = StandardScaler()
#     scaler.fit(X_train)
#     X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
#     X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_train.columns)
    

#     #clf_ridge = RandomForestClassifier(random_state=42) 
#     clf_ridge = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(100,))
#     clf_ridge.fit(X_train, y_train) 
#     pred = clf_ridge.predict(X_test)
#     print(classification_report(y_test, pred))
#     fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
#     ConfusionMatrixDisplay.from_predictions(y_test, pred, 
#                                             cmap='Blues', display_labels= ['not_churn', 'churn'], 
#                                             ax = ax, values_format = 'd')




## applying standard scaler
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)
dump(scaler, './model/scaler_jan22.joblib')


clf_all_mlp = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(100,))
clf_all_mlp.fit(X, y) 
dump(clf_all_mlp, './model/mlp_jan22.joblib')


In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
importances = pd.DataFrame({'feature':X.columns,'importance':np.round(clf_all.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances = importances[0:20]

In [ ]:
importances.reset_index().head(10)

In [ ]:
clf_all_mlp.coefs_

In [ ]:
df_fit = pd.concat([pd.DataFrame(promo_feature[p_type], columns=['variabel']), 
           pd.DataFrame(pd.Series(clf.coef_), columns=['bobot'])], 1)

In [ ]:
dump(clf_all, './model/rf_jan22.joblib')

In [ ]:
# dump(clf_all_mlp, './model/mlp_jan22.joblib')

In [ ]:
from sklearn.model_selection import StratifiedKFold
condition_2_3 = (
((df_tto_re_pv['TRO_NET 2021-09'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0)) | \
    
((df_tto_re_pv['TRO_NET 2021-09'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0)) |
    
((df_tto_re_pv['TRO_NET 2021-09'] == 0) & \
(df_tto_re_pv['TRO_NET 2021-10'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-11'] > 0) & \
(df_tto_re_pv['TRO_NET 2021-12'] == 0))
)

df_tto_re_pv['label'] = np.where(condition_2_3, 1, 0)


X = df_tto_re_pv.drop(
[
'TRO_MEMBERS',
'TRO_NET 2021-12', 
'TRO_DH_SPESIAL 2021-12',
'TRO_NO_ORDER 2021-12',
'SLA 2021-12',
'SLA_LATE 2021-12',
'RATING_BELOW 2021-12',
'RATING_AVG 2021-12',
'PV 2021-12',
'TRO_VOUCHER_USAGE 2021-12',
'TRO_PAYMENT_METHOD 2021-12',
'label'
], 1
)
y = df_tto_re_pv[['label']]



skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, y):
    print("="*50)
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]

    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0, stratify=y)
    
    ## applying standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_train.columns)
    

#     ## applying pca
#     n_comp = 5

#     pca = PCA(n_components=n_comp)
#     pca.fit(X_train)
#     X_train = pd.DataFrame(pca.transform(X_train), columns=(['col{}'.format(i) for i in range(1,n_comp+1)]))
#     X_test = pd.DataFrame(pca.transform(X_test), columns=(['col{}'.format(i) for i in range(1,n_comp+1)]))

    clf_ridge = RandomForestClassifier(random_state=42) 
#     clf_ridge = LinearSVC(C=0.01, penalty="l1", dual=False)
    clf_ridge.fit(X_train, y_train) 
    pred = clf_ridge.predict(X_test)
#     pred = (clf_ridge.predict_proba(X_test)[:,1] >= 0.5).astype(int)
    print(classification_report(y_test, pred))
    fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
    ConfusionMatrixDisplay.from_predictions(y_test, pred, 
                                            cmap='Blues', display_labels= ['not_churn', 'churn'], 
                                            ax = ax, values_format = 'd')



In [ ]:
df_tto_re_pv[df_tto_re_pv['label'] == 1].head(50)

In [ ]:
importances.head()

In [ ]:
importances = pd.DataFrame({'feature':X.columns,'importance':np.round(clf_ridge.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances = importances[0:20]
x_range = list(range(importances.shape[0]))
x_plot = list(importances.index)
y_plot = list(importances["importance"])
plt.figure(figsize=(30,13), dpi=250)
plt.title("Feature importances")
plt.bar(x_range, y_plot)
plt.xticks(x_range, x_plot, rotation='vertical', fontsize=18)
fig = plt.gcf()


In [ ]:
a = pd.DataFrame()

a['NET 11-10'] = df_tto_re_pv['TRO_NET 2021-12'] - df_tto_re_pv['TRO_NET 2021-11']
a['NET 10-09'] = df_tto_re_pv['TRO_NET 2021-11'] - df_tto_re_pv['TRO_NET 2021-10']
a['NET 11-09'] = df_tto_re_pv['TRO_NET 2021-12'] - df_tto_re_pv['TRO_NET 2021-10']

a['STRUK 11-10'] = df_tto_re_pv['TRO_NO_ORDER 2021-12'] - df_tto_re_pv['TRO_NO_ORDER 2021-11']
a['STRUK 10-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-11'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']
a['STRUK 11-09'] = df_tto_re_pv['TRO_NO_ORDER 2021-12'] - df_tto_re_pv['TRO_NO_ORDER 2021-10']

a['PV 11-10'] = df_tto_re_pv['PV 2021-12'] - df_tto_re_pv['PV 2021-11']
a['PV 10-09'] = df_tto_re_pv['PV 2021-11'] - df_tto_re_pv['PV 2021-10']
a['PV 11-09'] = df_tto_re_pv['PV 2021-12'] - df_tto_re_pv['PV 2021-10']

a['SLA 11-10'] = df_tto_re_pv['SLA 2021-12'] - df_tto_re_pv['SLA 2021-11']
a['SLA 10-09'] = df_tto_re_pv['SLA 2021-11'] - df_tto_re_pv['SLA 2021-10']
a['SLA 11-09'] = df_tto_re_pv['SLA 2021-12'] - df_tto_re_pv['SLA 2021-10']

a['BASKET 11'] = df_tto_re_pv['TRO_NET 2021-12']/df_tto_re_pv['TRO_NO_ORDER 2021-12']
a['BASKET 10'] = df_tto_re_pv['TRO_NET 2021-11']/df_tto_re_pv['TRO_NO_ORDER 2021-11']
a['BASKET 09'] = df_tto_re_pv['TRO_NET 2021-10']/df_tto_re_pv['TRO_NO_ORDER 2021-10']

a['BASKET AVG'] = (df_tto_re_pv['BASKET 11'] + df_tto_re_pv['BASKET 10'] + df_tto_re_pv['BASKET 09']) / 3

In [ ]:
X_12 = df_tto_re_pv.drop(
[
'TRO_MEMBERS',
'TRO_NET 2021-10', 
'TRO_DH_SPESIAL 2021-10',
'TRO_NO_ORDER 2021-10',
'SLA 2021-10',
'SLA_LATE 2021-10',
'RATING_BELOW 2021-10',
'RATING_AVG 2021-10',
'PV 2021-10',
'TRO_VOUCHER_USAGE 2021-10',
'TRO_PAYMENT_METHOD 2021-10',
# 'label'
], 1
)

# X_12 = pd.concat([X_12.iloc[:,0:-16],a],1)
# X_12 = X_12.fillna(0)
# y_12 = df_tto_re_pv['label']

In [ ]:
# X_12.head()

In [ ]:
scaler_all = load('./model/scaler_jan22.joblib')

In [ ]:
clf_all = load('./model/mlp_jan22.joblib')

In [ ]:
X_12_scale = pd.DataFrame(scaler_all.transform(X_12), index=X_12.index, columns=X_12.columns)
pred = (clf_all.predict_proba(X_12_scale)[:,1] >= 0.6).astype(int)

In [ ]:
df_mem_12 = pd.concat([df_tto_re_pv[['TRO_MEMBERS']].reset_index(drop=True)
                       , pd.DataFrame(pred, columns=['PRED'])], 1)

In [ ]:
df_mem_12[df_mem_12['PRED'] == 1].shape

In [ ]:
df_churn_sel = df_mem_12[df_mem_12['PRED'] == 1].sample(5000)

In [ ]:
df_churn_sel.to_csv('./out_member/member_churn_feb.csv', index=False)

In [ ]:
pd.merge(df_churn_sel,df_tto_re_pv,on='TRO_MEMBERS', how='left').head(1000)